In [14]:


import os

whereami = os.environ.get("WHEREAMI")

if whereami=="shane-local":
    print(whereami)
elif whereami=="databricks-sandbox":
    %pip install dbutils
    print(whereami)
    dbutils.secrets.listScopes()
    dbutils.library.restartPython()
elif whereami=="databricks-prod":
    %pip install dbutils
    print(whereami)
    dbutils.secrets.listScopes()
    dbutils.library.restartPython()
    


Note: you may need to restart the kernel to use updated packages.


In [ ]:
%pip install -q torch 
%pip install -q transformers 
%pip install -q accelerate 
%pip install -q bitsandbytes 
%pip install -q langchain 
%pip install -q sentence-transformers 
%pip install -q faiss-gpu 
%pip install -q openpyxl 
%pip install -q pacmap